Скачиваем компилатор

In [2]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vqr4fgjx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vqr4fgjx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=5c0f42b8ae0a3adadbae6ab98085e0a550981be527c5d5a9f304ed6e7a8b74d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-mrqwyzy6/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Непосредственно программа

In [69]:
%%cu
#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <stdexcept>
using namespace std;
/*
 Умножение матриц a(N1, M1) и b(N2, M2)
*/
__global__ void kernel(double* c, double* a, double* b, int N1, int M1, int N2, int M2) {
	//мысленно дробим матрицу на подматрицы размерами blockDim.x X blockDim.y и каждому потоку в блоке даем свой элемент результирующей матрицы
  int ROW = blockIdx.x*blockDim.x+threadIdx.x;
  int COL = blockIdx.y*blockDim.y+threadIdx.y;

  //проверяем что мы не вышли за размер матрицы
  if (COL < M2 && ROW < N1) {
      double sum  = 0;
      for (int i = 0; i < M1; ++i) {
          //printf("(%d,%d) += a[%d]*b[%d]\n", ROW, COL, ROW*M1 + i, N2 * i + COL);
          //считаем рехультирующий элемент перемножением ROW строки на COL столбец
          sum = sum+ a[ROW*M1 + i] * b[M2 * i + COL];
      }
      c[ROW * M2 + COL] = sum;
  }
}

void handle_cuda_result(cudaError_t cuerr, char msg[]) {
    if (cuerr != cudaSuccess) {
        fprintf(stderr, cudaGetErrorString(cuerr));
        fprintf(stderr, "\n");
        throw runtime_error(msg);
    }
}

/*
 Загрузка данных на GPU
 @ resultGpuPointer - перезаписывается, указатель на указатель на память GPU (чтобы значение указателя сохранялось)
 @ mat - указатель на массив с развернутой матрицей
 @ size - размер развернутой матрицы
*/
void upload_to_device(double** resultGpuPointer, double* mat, int size) {
    int sizeInBytes = size * sizeof(double);
    //выделяем память
    cudaError_t cuerr = cudaMalloc((void**)resultGpuPointer, sizeInBytes);
    handle_cuda_result(cuerr, "Cannot allocate device array");

    // копируем массив
    //*resultGpuPointer - разыменовываем указатель на указатель, чтобы передать указатель
    cuerr = cudaMemcpy(*resultGpuPointer, mat, sizeInBytes, cudaMemcpyHostToDevice);
    handle_cuda_result(cuerr, "Cannot copy a array from host to device");
}

/*
Загрузка данных на хост-машину с GPU
 @ resultGpuPointer - указатель на память GPU
 @ gpuMatPointer - указатель на массив с развернутой матрицей, куда будет загружена матрица (память должная быть выделена)
 @ size - размер развернутой матрицы
*/
void download_from_device(double* gpuMatPointer, double* resultMat, int size) {
    int sizeInBytes = size * sizeof(double);
    // копируем массив
    cudaError_t cuerr = cudaMemcpy(resultMat, gpuMatPointer, sizeInBytes, cudaMemcpyDeviceToHost);
    handle_cuda_result(cuerr, "Cannot copy a array from device to host");
}

/*
  Умножение матриц a(N1, M1) и b(N2, M2)
*/
double mat_mul_gpu(double* a, double* b, double* c, int N1, int M1, int N2, int M2, dim3 blocksPerGrid, dim3 threadsPerBlock){
    cudaError_t cuerr;
    // Выделение памяти на устройстве
    double* aGpuPointer = NULL;

    //передаем указатель на указатель, чтобы работать со значением указателя                                                                                                                ///
    upload_to_device(&aGpuPointer, a, N1*M1);
    double* bGpuPointer = NULL;
    upload_to_device(&bGpuPointer, b, N2*M2);
    double* cGpuPointer = NULL;
    upload_to_device(&cGpuPointer, c, N1*M2);

    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cuerr = cudaEventCreate(&start);
    handle_cuda_result(cuerr, "Cannot create CUDA start event");

    cuerr = cudaEventCreate(&stop);
    handle_cuda_result(cuerr, "Cannot create CUDA stop event");

    cuerr = cudaEventRecord(start, 0);
    handle_cuda_result(cuerr, "Cannot record CUDA start event");

    //Запуск ядра
    kernel <<< blocksPerGrid, threadsPerBlock >>> (cGpuPointer, aGpuPointer, bGpuPointer, N1, M1, N2, M2);
    handle_cuda_result(cudaGetLastError(), "Cannot launch CUDA kernel");

    // Синхронизация устройств
    cuerr = cudaDeviceSynchronize();
    handle_cuda_result(cudaGetLastError(), "Cannot synchronize CUDA kernel");

    // Установка точки окончания
    cuerr = cudaEventRecord(stop, 0);
    handle_cuda_result(cuerr, "Cannot record CUDA stop event");

    // Копирование результата на хост
    download_from_device(cGpuPointer, c, N1*M2);

    cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
    handle_cuda_result(cuerr, "Cannot get elapsed time");
    double time = gpuTime/ 1000;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(aGpuPointer);
    cudaFree(bGpuPointer);
    cudaFree(cGpuPointer);
    return time;
}


void count_cuda_dims(dim3& blocksPerGrid, dim3& threadsPerBlock, int N1, int M1, int N2, int M2) {
    //максимум потоков в блоке - 1024
    int xthreadsPerBlock = N1 < 32 ? N1 : 32;
    int ythreadsPerBlock = M2 < 32 ? M2 : 32;
    int xblocksPerGrid = ceil((float)N1 / xthreadsPerBlock);
    int yblocksPerGrid = ceil((float)M2 / ythreadsPerBlock);
    printf("grid(%d, %d) block(%d,%d)\n", xblocksPerGrid, yblocksPerGrid, xthreadsPerBlock, ythreadsPerBlock);
    blocksPerGrid =  dim3(xblocksPerGrid, yblocksPerGrid, 1);
    threadsPerBlock =  dim3(xthreadsPerBlock, ythreadsPerBlock, 1);

}


double mat_mul_cpu(double* a, double* b, double* c, int N1, int M1, int N2, int M2){
  double time = clock();
	for (int i = 0; i < N1; ++i) {
    for (int j = 0; j < M2; ++j) {
		  double sum = 0;
      for (int k = 0; k < M1; ++k) {
        sum = sum+ a[i*M1 + k] * b[M2 * k + j];
		  }
		  c[i * M2 + j] = sum;
    }
  }
  time = clock() - time;
  time/=CLOCKS_PER_SEC;
  return time;
}

void fill_mat(double* mat, int n, int m) {
    srand (time(NULL));
    double re, im;
    for(int i = 0; i < n * m; ++i) {
        mat[i] = rand()/10000000.0;
    }

}

void check_mat(double* cpu, double* gpu,  int n, int m, double precision) {
  for (int i = 0; i < n; ++i) {
    for (int j = 0; j < m; ++j) {
      if (fabs(cpu[i*m+j] - gpu[i*m+j]) > precision) {
          fprintf(stderr, "Матрицы не равны: cpu(%d,%d)=%f; gpu(%d,%d)=%f\n", i,j, cpu[i*m+j], i,j, gpu[i*m+j]);
          throw runtime_error("Матрицы не равны");
      }
    }
  }
}

int main(int argc, char* argv[]) {
    int an = 2048; int am = 2048;
    int bn = 2048; int bm = 2048;
    int cn = an; int cm = bm;
    double* a = (double*) malloc(an*am*sizeof(double));
    double* b = (double*) malloc(bn*bm*sizeof(double));
    double* c_gpu = (double*) malloc(cn*cm*sizeof(double));
	  double* c_cpu = (double*) malloc(cn*cm*sizeof(double));

    fill_mat(a, an, am);
    fill_mat(b, bn, bm);

    dim3 threadsPerBlock;
    dim3 blocksPerGrid;

    count_cuda_dims(blocksPerGrid, threadsPerBlock, an,am,bn,bm);

    double timeGpu = mat_mul_gpu(a,b,c_gpu, an,am,bn,bm, blocksPerGrid, threadsPerBlock);
    printf("GPU: %f\n", timeGpu);

    double timeCpu = mat_mul_cpu(a,b,c_cpu, an,am,bn,bm);
    printf("CPU: %f\n", timeCpu);

    check_mat(c_cpu, c_gpu, cn, cm, 0.000001);

    return 0;

}

grid(64, 64) block(32,32)
GPU: 0.475556
CPU: 135.858048

